In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/3rd-ml100marathon-midterm/train_data.csv
/kaggle/input/3rd-ml100marathon-midterm/sample_submission.csv
/kaggle/input/3rd-ml100marathon-midterm/enron61702insiderpay.pdf
/kaggle/input/3rd-ml100marathon-midterm/test_features.csv


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import warnings
warnings.filterwarnings('ignore')


In [3]:
data_path = '/kaggle/input/3rd-ml100marathon-midterm/'
df_train = pd.read_csv(data_path + 'train_data.csv')

df_test = pd.read_csv(data_path + 'test_features.csv')

df_train.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [4]:
df_train.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,6.100000e+01,2.800000e+01,3.400000e+01,13.000000,8.100000e+01,73.000000,65.000000,65.000000,65.000000,2.000000e+00,4.900000e+01,6.900000e+01,8.200000e+01,10.000000,7.300000e+01,65.000000,65.000000,9.600000e+01,9.800000e+01
mean,1.147436e+06,6.344374e+05,-4.625664e+05,89397.846154,2.985081e+06,51040.547945,711.323077,64.800000,40.092308,4.096250e+07,7.926171e+05,4.471774e+05,1.294855e+06,-221885.700000,2.739025e+05,1111.369231,2156.061538,2.590977e+06,3.527136e+06
std,1.505189e+06,8.603646e+05,8.095392e+05,41143.391399,6.004174e+06,47596.682104,2074.497628,91.863214,88.901407,5.736404e+07,9.504645e+05,1.341564e+06,2.498335e+06,205191.374121,1.716647e+05,1165.852016,2811.676718,1.056645e+07,7.182997e+06
min,7.000000e+04,-1.025000e+05,-3.504386e+06,3285.000000,3.285000e+03,148.000000,12.000000,0.000000,0.000000,4.000000e+05,7.102300e+04,2.000000e+00,4.409300e+04,-560222.000000,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.500000e+05,7.656750e+04,-5.527032e+05,101250.000000,4.004780e+05,18834.000000,19.000000,10.000000,0.000000,2.068125e+07,2.750000e+05,9.720000e+02,2.689220e+05,-389621.750000,2.061210e+05,178.000000,517.000000,3.024025e+05,4.211518e+05
50%,7.500000e+05,1.951900e+05,-1.175340e+05,108579.000000,8.500100e+05,41953.000000,45.000000,28.000000,7.000000,4.096250e+07,4.221580e+05,5.238200e+04,4.628225e+05,-139856.500000,2.516540e+05,599.000000,1088.000000,1.106740e+06,9.979710e+05
75%,1.000000e+06,8.342052e+05,-2.708325e+04,112492.000000,2.165172e+06,59175.000000,215.000000,88.000000,27.000000,6.124375e+07,8.318090e+05,3.620960e+05,9.664905e+05,-77953.250000,2.885890e+05,1902.000000,2649.000000,1.985668e+06,2.493616e+06
max,8.000000e+06,2.964506e+06,-1.042000e+03,125034.000000,3.434838e+07,228763.000000,14368.000000,528.000000,411.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,44093.000000,1.111258e+06,4527.000000,15149.000000,1.035598e+08,4.911008e+07


In [5]:
df_train['poi'] = df_train['poi'].astype(float)
train_Y = df_train['poi']
ids = df_test['name']

df_train = df_train.drop(['name', 'email_address', 'poi'] , axis=1)
df_test = df_test.drop(['name', 'email_address'] , axis=1)

df = pd.concat([df_train,df_test])
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 32
Data columns (total 19 columns):
bonus                        146 non-null float64
deferral_payments            146 non-null float64
deferred_income              146 non-null float64
director_fees                146 non-null float64
exercised_stock_options      146 non-null float64
expenses                     146 non-null float64
from_messages                146 non-null float64
from_poi_to_this_person      146 non-null float64
from_this_person_to_poi      146 non-null float64
loan_advances                146 non-null float64
long_term_incentive          146 non-null float64
other                        146 non-null float64
restricted_stock             146 non-null float64
restricted_stock_deferred    146 non-null float64
salary                       146 non-null float64
shared_receipt_with_poi      146 non-null float64
to_messages                  146 non-null float64
total_payments               146 non-null floa

In [6]:
#finalcial 和 email 數據的數值資訊都捨去, 有就是1, 沒有(na)就是0
for col in df:
    df[col] = df[col].apply(lambda x: 1 if abs(x)>0 else 0)

#先前測試, 發現有時候把 'bonus' 和 'deferred_income' 結合成新的 feature 結果比較好
df ['com2'] = df['bonus'] + df['deferred_income']
df.drop(['bonus', 'deferred_income'], axis = 1)
        
df.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,com2
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,0.561644,0.267123,0.335616,0.116438,0.698630,0.650685,0.589041,0.506849,0.452055,0.027397,0.452055,0.636986,0.753425,0.123288,0.650685,0.589041,0.589041,0.856164,0.863014,0.897260
std,0.497894,0.443980,0.473831,0.321854,0.460433,0.478395,0.493701,0.501674,0.499409,0.163800,0.499409,0.482524,0.432501,0.329899,0.478395,0.493701,0.493701,0.352131,0.345016,0.749524
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [7]:
#測試幾種model後, 發現 gdbt 和 clf 分的比較好

gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=9,
                                  max_depth=6, learning_rate=0.03)
         
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]
            
X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.25, random_state=4)
          
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
gdbt_score = cross_val_score(gdbt, train_X, train_Y, cv=5).mean()

sub = pd.DataFrame({'name': ids, 'poi': gdbt_pred})
sub.to_csv('mid_abs_com2_gdbt1027.csv', index=False)
            

# 建立模型
clf = DecisionTreeClassifier()
clf.fit(train_X, train_Y)
clf_pred = clf.predict_proba(test_X)[:,1]
clf_score = cross_val_score(clf, train_X, train_Y, cv=5).mean()

sub = pd.DataFrame({'name': ids, 'poi': clf_pred})
sub.to_csv('mid_abs_com2_clf1027.csv', index=False)


print('gdbt:')
print(gdbt_score)

print('clf:')
print(clf_score)            



gdbt:
0.8679841897233201
clf:
0.8328063241106719


gdbt_ef:
0.8679841897233201

clf_ef:
0.8588932806324111